In [1]:
####Main Algorithm is copied from the following places 
################################################################*************************************************
#L. A. Gatys, A. S. Ecker, and M. Bethge, "Image style transfer using convolutional neural networks," in Proceedings of the IEEE conference on computer vision and pattern recognition, 2016, pp. 2414-2423. 
#A. K. Mallik. "Neural Style Transfer Using PyTorch." https://towardsdatascience.com/implementing-neural-style-transfer-using-pytorch-fd8d43fb7bfa (accessed.
#A. Ecker, A. Bethge, and L. Gatys, "A neural algorithm of artistic style," arXiv preprint arXiv:1508.06576, 2015. 


##########################****************************************************************

#### Combined 2 style images and obtained the results 
#### Experiments done on different convolutional layers

In [2]:

#importing the library 
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image

In [3]:
#Loadung the model vgg19 that will serve as the base model
base_model=models.vgg19(pretrained=True).features
#Assigning the GPU to the variable device
device=torch.device( "cuda" if (torch.cuda.is_available()) else 'cpu')

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [4]:
def image_loader(path):
    img = Image.open(path)
    #resizing the image and convert image into a tensor 
    loader=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    #Adding an extra dimension at 0th index, for batch size 
    img = loader(img).unsqueeze(0)
    return img.to(device, torch.float)

#loading the content image and 2 style images 
content_image = image_loader('Content_Img.jpg')
style_image1=image_loader('Style_Image.jpg')
style_image2=image_loader('Style_Image_1.jpg')

#cloning the content image 
gen_image = content_image.clone().requires_grad_(True)


In [5]:
#Defining a class that for the model
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        #'conv1_1',''conv2_1','conv3_1','conv4_1','conv5_1'
        self.req_features= ['0','5','10','19','28'] #Original 
       #self.req_features= ['1','5','10','19','28'] #trial 1
       #self.req_features= ['0','5','10','11','13'] #trial 2
       #self.req_features= ['0','5','11','19','28'] #trial 3
       #self.req_features= ['20','29','12','6','0'] #trial 4
        #Since we need only the 5 layers in the model so we will be dropping all the rest layers from the features of the model
        self.model=models.vgg19(pretrained=True).features[:29] #model will contain the first 29 layers
    
   
    #x holds the input tensor(image) that will be feeded to each layer
    def forward(self,x):
        #initialize an array that wil hold the activations from the chosen layers
        features=[]
        #Iterate over all the layers of the mode
        for layer_num,layer in enumerate(self.model):
            #activation of the layer will stored in x
            x=layer(x)
            #appending the activation of the selected layers and return the feature array
            if (str(layer_num) in self.req_features):
                features.append(x)
                
        return features

In [6]:
def calc_content_loss(gen_feat,orig_feat):
    #calculating the content loss of each layer by calculating the MSE between the content and generated features and adding it to content loss
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

In [7]:
def calc_style_loss(gen,style):
    #Calculating the gram matrix for the style and the generated image
    batch_size,channel,height,width=gen.shape

    G=torch.mm(gen.view(channel,height*width),gen.view(channel,height*width).t())
    A=torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())
        
    #Calcultating the style loss of each layer by calculating the MSE between the gram matrix of the style image and the generated image and adding it to style loss
    style_l=torch.mean((G-A)**2)
    return style_l

In [8]:
def calculate_loss(gen_features, orig_feautes, style_featues1, style_featues2):
    style_loss1=content_loss=style_loss2=0
    for gen,cont,style1,style2 in zip(gen_features,orig_feautes,style_featues1, style_featues2):
        #extracting the dimensions from the generated image
        content_loss+=calc_content_loss(gen,cont)
        style_loss1+=calc_style_loss(gen,style1)
        style_loss2+=calc_style_loss(gen,style2)
    
    #calculating the total loss of e th epoch
    total_loss=alpha*content_loss + beta*style_loss1 + beta*style_loss2 
    return total_loss

In [9]:
#Load the model to the GPU
base_model=VGG().to(device).eval() 

#initialize the paramerters required for fitting the model
epoch=7000
lr=0.004
alpha=8
beta=70

#using adam optimizer and it will update the generated image not the model parameter 
optimizer=optim.Adam([gen_image],lr=lr)

In [ ]:
#iterating for 1000 times
for e in range (epoch):
    #extracting the features of generated, content and the original required for calculating the loss
    gen_features=base_model(gen_image)
    orig_feautes=base_model(content_image)
    style_featues1=base_model(style_image1)
    style_featues2=base_model(style_image2)
    
    #iterating over the activation of each layer and calculate the loss and add it to the content and the style loss
    total_loss=calculate_loss(gen_features, orig_feautes, style_featues1,style_featues2 )
    #optimize the pixel values of the generated image and backpropagate the loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    #print the image and save it after each 100 epoch
    if(e/100):
        print(total_loss)
        
        save_image(gen_image,"gen.png")  